In [1]:
# Author: Vanshika Gupta
# Jointly with: Prof Chrysafis Vogiatzis

In [2]:
from gurobipy import *
import matplotlib.pyplot as plt

#### Data

In [3]:
# number of facilities
nFacilities = 6
nCustomers = 10

# cost to send item from warehouse to customer y-> customer, x-> facility
cost = [(3,5,8,12,10,11), (6,7,10,12,8,10), (5,3,8,10,12,10), (7,6,7,10,9,5), (10,8,5,8,6,5), (12,6,4,7,5,4),
        (7,10,6,6,8,3), (5,6,9,4,4,7), (8,10,7,5,4,6), (10,12,8,3,4,6) ] 

# cost to open facility
fixedCost = [1750000, 2000000, 2500000, 2250000, 1500000, 1000000] 

#facility capacity
capacity = [75, 50, 120, 100, 60, 50]

# customer demand
demand = [25, 15, 20, 10, 15, 15, 25, 20, 30, 20]

#### Variables

In [4]:
# number of items shipped from warehouse to customer y-> customer, x-> facility
itemCount = {}

# whether facility is open or closed
y = {}

# whether facility is supplying customer or not
z = {}

#### Creating a model and Add Variables

In [5]:
model=Model("facilities")

y = model.addVars(nFacilities, vtype = GRB.BINARY)
itemCount = model.addVars(nCustomers, nFacilities, vtype = GRB.INTEGER)
z = model.addVars(nCustomers, nFacilities, vtype = GRB.BINARY)

Academic license - for non-commercial use only - expires 2022-09-08
Using license file /home/vanshika/gurobi.lic


#### Adding constraint

In [6]:
# number of open facilities is less than or equal to total
model.addConstr(quicksum(y) <= nFacilities)

for i in range(nFacilities):
    expr = 0
    for j in range(nCustomers):
        expr += itemCount[j,i]
    model.addConstr(expr <= capacity[i]*y[i])
    
for i in range(nCustomers):
    expr = 0
    for j in range(nFacilities):
        expr += itemCount[i,j]*z[i,j]
    model.addConstr(expr == demand[i])
    
for i in range(nCustomers):
    expr = 0
    for j in range(nFacilities):
        expr += z[i,j]
    model.addConstr(expr == 1)

#### Set Objective

In [7]:
obj = 0
for i in range(nFacilities):
    for j in range(nCustomers):
        obj += cost[j][i]*itemCount[j,i]
    obj += fixedCost[i]*y[i]
    
model.setObjective(obj)

In [8]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 17 rows, 126 columns and 132 nonzeros
Model fingerprint: 0xb1541b71
Model has 10 quadratic constraints
Variable types: 0 continuous, 126 integer (66 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e+00, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
  QRHS range       [1e+01, 3e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 206 rows, 186 columns, 606 nonzeros
Variable types: 0 continuous, 186 integer (66 binary)
Found heuristic solution: objective 1.100074e+07
Found heuristic solution: objective 6751255.0000
Found heuristic solution: objective 6500860.0000

Root relaxation: objective 4.063665e+06, 22 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl U

#### Print Result

In [9]:
print("Facility\t Open/Closed")
for i in range(nFacilities):
    print(i+1,'\t\t', round(y[i].X))
print('\n')
    
print("Customer\tItemCount for Facility (F)\n")
print('\t\t F1\t F2\t F3\t F4\t F5\t F6 \n',)
for i in range(nCustomers):
    print('Customer', i+1, end='')
    for j in range(nFacilities):
        print('\t', round(itemCount[i,j].X), end='')
    print('\n')

Facility	 Open/Closed
1 		 1
2 		 0
3 		 1
4 		 0
5 		 0
6 		 0


Customer	ItemCount for Facility (F)

		 F1	 F2	 F3	 F4	 F5	 F6 

Customer 1	 25	 0	 0	 0	 0	 0

Customer 2	 15	 0	 0	 0	 0	 0

Customer 3	 20	 0	 0	 0	 0	 0

Customer 4	 0	 0	 10	 0	 0	 0

Customer 5	 0	 0	 15	 0	 0	 0

Customer 6	 15	 0	 0	 0	 0	 0

Customer 7	 0	 0	 25	 0	 0	 0

Customer 8	 0	 0	 20	 0	 0	 0

Customer 9	 0	 0	 30	 0	 0	 0

Customer 10	 0	 0	 20	 0	 0	 0



In [10]:
print("Min Total Cost incurred: $", model.getObjective().getValue())

Min Total Cost incurred: $ 4251290.0
